In [1]:
import pandas as pd
train = pd.read_csv('./data/train_20171215.txt',sep='\t')
new_df = train.drop(['day_of_week', 'brand'],axis=1).groupby('date').sum()
new_df['date'] = new_df.index
new_df = pd.merge(new_df, train[['date','day_of_week']].drop_duplicates(), how='left', on='date')
print new_df.shape
new_df.to_csv('./data/train_group.txt', sep='\t')
new_df.head(5)

(1032, 3)


,cnt,date,day_of_week
0,68,1,3
1,36,2,4
2,5565,3,5
3,4966,4,6
4,3346,5,7


In [2]:
# 通过 day_of_week 发现日期数据并不连续
# 为了做时序预测，填充缺失数据，暂时填充为0后续再一起处理
def fill_missing(df):
    modi_df = pd.DataFrame(columns=['date', 'day_of_week', 'cnt'], dtype=int)
    last_day_of_week = 0
    for index, row in df.iterrows():
        now_day_of_week = row['day_of_week']
        # 若 day_of_week 当前天减上一天不是1 或 -6，说明有跳跃
        while (last_day_of_week != 0) and (now_day_of_week - last_day_of_week != 1) and (now_day_of_week - last_day_of_week != -6):
            if last_day_of_week == 7:
                last_day_of_week = 1
            else:
                last_day_of_week += 1
            modi_df = modi_df.append({'day_of_week': last_day_of_week, 'cnt': 0}, ignore_index=True)
        modi_df = modi_df.append(row, ignore_index=True)
        last_day_of_week = now_day_of_week
    return modi_df

In [3]:
modi_df = fill_missing(new_df)
print modi_df.shape
# 改变 cnt 列的值属性为数值型
modi_df.cnt = pd.to_numeric(modi_df.cnt)
# print modi_df.dtypes
modi_df.head(15)

(1192, 3)


,date,day_of_week,cnt
0,1,3,68
1,2,4,36
2,3,5,5565
3,4,6,4966
4,5,7,3346
5,6,1,3396
6,7,2,4146
7,8,3,3096
8,9,4,2713
9,10,5,2409


In [4]:
# import warnings 
# warnings.filterwarnings("ignore")
for i in range(1, 8):
    # 对每个 day_of_week 依次填充
    day_of_week_mean = int(new_df[new_df.day_of_week == i]['cnt'].mean())
    modi_df.loc[modi_df.day_of_week == i, 'cnt']= modi_df[modi_df.day_of_week == i].cnt.mask(modi_df.cnt == 0, day_of_week_mean)
print modi_df.head(15)

   date day_of_week   cnt
0     1           3    68
1     2           4    36
2     3           5  5565
3     4           6  4966
4     5           7  3346
5     6           1  3396
6     7           2  4146
7     8           3  3096
8     9           4  2713
9    10           5  2409
10   11           6   275
11  NaN           7   742
12   12           1  2735
13   13           2  2386
14   14           3  1901


In [5]:
modi_df['ds'] = pd.date_range(start = '1/2/2013', periods=1192)
# modi_df[(modi_df['ds'] >= '10/1/2014') & (modi_df['ds'] <= '10/7/2014')]
modi_df.tail()

,date,day_of_week,cnt,ds
1187,NaN,7,742,2016-04-03
1188,NaN,1,2364,2016-04-04
1189,1030,2,4003,2016-04-05
1190,1031,3,2513,2016-04-06
1191,1032,4,1306,2016-04-07


In [6]:
import numpy as np

prophet_train = modi_df.loc[:,['ds', 'cnt']]
prophet_train.rename(columns = {'cnt':'y'}, inplace = True)
# prophet_train.y = np.log1p(prophet_train.y)
prophet_train.head()

,ds,y
0,2013-01-02,68
1,2013-01-03,36
2,2013-01-04,5565
3,2013-01-05,4966
4,2013-01-06,3346


## 构造模型

In [8]:
from fbprophet import Prophet
import pickle as pkl
with open('./data/holidays.pkl', 'rb') as rf:
    holidays = pkl.load(rf)
m = Prophet(holidays = holidays)

In [9]:
def evaluate_model(forecast,test):
    '''
    计算模型mse
    '''
    assert (test.ds == forecast.ds).all()
    mse = sum((test.y-forecast.yhat)**2)
    mse = float(mse)/len(test)
    return mse
    

In [10]:
#测试
train_d = prophet_train[:1000]
test_d = prophet_train[1000:]

In [12]:
test_d.ds

1000   2015-09-29
1001   2015-09-30
1002   2015-10-01
1003   2015-10-02
1004   2015-10-03
1005   2015-10-04
1006   2015-10-05
1007   2015-10-06
1008   2015-10-07
1009   2015-10-08
1010   2015-10-09
1011   2015-10-10
1012   2015-10-11
1013   2015-10-12
1014   2015-10-13
1015   2015-10-14
1016   2015-10-15
1017   2015-10-16
1018   2015-10-17
1019   2015-10-18
1020   2015-10-19
1021   2015-10-20
1022   2015-10-21
1023   2015-10-22
1024   2015-10-23
1025   2015-10-24
1026   2015-10-25
1027   2015-10-26
1028   2015-10-27
1029   2015-10-28
          ...    
1162   2016-03-09
1163   2016-03-10
1164   2016-03-11
1165   2016-03-12
1166   2016-03-13
1167   2016-03-14
1168   2016-03-15
1169   2016-03-16
1170   2016-03-17
1171   2016-03-18
1172   2016-03-19
1173   2016-03-20
1174   2016-03-21
1175   2016-03-22
1176   2016-03-23
1177   2016-03-24
1178   2016-03-25
1179   2016-03-26
1180   2016-03-27
1181   2016-03-28
1182   2016-03-29
1183   2016-03-30
1184   2016-03-31
1185   2016-04-01
1186   201

In [13]:
future = pd.DataFrame({'ds':pd.date_range(start='9/29/2015',periods=192)})

In [15]:
train_d

,ds,y
0,2013-01-02,68
1,2013-01-03,36
2,2013-01-04,5565
3,2013-01-05,4966
4,2013-01-06,3346
5,2013-01-07,3396
6,2013-01-08,4146
7,2013-01-09,3096
8,2013-01-10,2713
9,2013-01-11,2409


In [16]:
m.fit(train_d)

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [17]:
forecast = m.predict(future)

In [18]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
187,2016-04-03,786.510863,-28.669805,1592.100111
188,2016-04-04,2305.071746,1472.980086,3146.963876
189,2016-04-05,2363.369594,1493.837197,3145.133499
190,2016-04-06,2031.439913,1202.776220,2851.963455
191,2016-04-07,1594.705132,714.987555,2427.609661


In [21]:
forecast

,ds,trend,trend_lower,trend_upper,yhat_lower,yhat_upper,holiday_2013,holiday_2013_lower,holiday_2013_upper,holiday_2014,...,seasonalities,seasonalities_lower,seasonalities_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,yhat
0,2015-09-29,1798.124102,1798.124102,1798.124102,1844.185242,3512.215426,0.0,0.0,0.0,0.0,...,908.732902,908.732902,908.732902,734.291585,734.291585,734.291585,174.441317,174.441317,174.441317,2706.857004
1,2015-09-30,1798.497334,1798.497334,1798.497334,1580.210494,3232.211470,0.0,0.0,0.0,0.0,...,592.763533,592.763533,592.763533,414.526423,414.526423,414.526423,178.237110,178.237110,178.237110,2391.260866
2,2015-10-01,1798.870566,1798.870566,1798.870566,-494.968641,1132.744945,0.0,0.0,0.0,0.0,...,169.517063,169.517063,169.517063,-12.050168,-12.050168,-12.050168,181.567231,181.567231,181.567231,306.599502
3,2015-10-02,1799.243797,1799.243797,1799.243797,979.477170,2491.562318,0.0,0.0,0.0,0.0,...,402.040233,402.040233,402.040233,217.513143,217.513143,217.513143,184.527090,184.527090,184.527090,1766.798328
4,2015-10-03,1799.617029,1799.617029,1799.617029,-373.891252,1186.433435,0.0,0.0,0.0,0.0,...,-958.427726,-958.427726,-958.427726,-1145.608252,-1145.608252,-1145.608252,187.180526,187.180526,187.180526,406.703601
5,2015-10-04,1799.990261,1799.990261,1799.990261,-145.853167,1479.667701,0.0,0.0,0.0,0.0,...,-681.416067,-681.416067,-681.416067,-870.971525,-870.971525,-870.971525,189.555458,189.555458,189.555458,684.088492
6,2015-10-05,1800.363493,1800.363493,1800.363493,1501.562190,3046.593518,0.0,0.0,0.0,0.0,...,853.939775,853.939775,853.939775,662.298794,662.298794,662.298794,191.640981,191.640981,191.640981,2219.817565
7,2015-10-06,1800.736724,1800.736724,1800.736724,1470.445589,3113.649062,0.0,0.0,0.0,0.0,...,927.677626,927.677626,927.677626,734.291585,734.291585,734.291585,193.386041,193.386041,193.386041,2293.928648
8,2015-10-07,1801.109956,1801.109956,1801.109956,1228.986042,2847.290585,0.0,0.0,0.0,0.0,...,609.226214,609.226214,609.226214,414.526423,414.526423,414.526423,194.699792,194.699792,194.699792,1975.850468
9,2015-10-08,1801.483188,1801.483188,1801.483328,2341.664393,4021.948966,0.0,0.0,0.0,0.0,...,183.403476,183.403476,183.403476,-12.050168,-12.050168,-12.050168,195.453644,195.453644,195.453644,3212.189088


In [26]:
test_d.reset_index(drop=True,inplace=True)

In [27]:
test_d

,ds,y
0,2015-09-29,3804
1,2015-09-30,3046
2,2015-10-01,1625
3,2015-10-02,13
4,2015-10-03,412
5,2015-10-04,15
6,2015-10-05,2364
7,2015-10-06,2495
8,2015-10-07,88
9,2015-10-08,4354


# 测试评估模型

In [30]:
evaluate_model(forecast,test_d)

734674.8004211477

# 根据brand区分数据，依次进行预测、合并预测结果、评估模型

In [33]:
df = pd.read_csv('./data/train_20171215.txt',sep='\t')
df_group = df.groupby('brand')
for num,d in df_group:
    print d

      date  day_of_week  brand   cnt
0        1            3      1    20
2        2            4      1    16
4        3            5      1  1411
9        4            6      1  1176
14       5            7      1   812
19       6            1      1   889
24       7            2      1   893
29       8            3      1   569
34       9            4      1   623
39      10            5      1   470
44      11            6      1    87
48      12            1      1   575
53      13            2      1   544
58      14            3      1   305
63      15            4      1   735
68      16            5      1   627
73      17            6      1    65
78      19            1      1   819
83      20            2      1   610
88      21            3      1   447
93      22            4      1   447
98      23            5      1   192
103     24            6      1    22
108     25            1      1   767
113     26            2      1   697
118     27            3      1   508
1

In [96]:
def fill_missing(df,day=1192):
    '''
    存在每个brand开头缺失和结尾缺失的情况
    '''
    modi_df = pd.DataFrame(columns=['date', 'day_of_week', 'cnt'], dtype=int)
    last_day_of_week = 0
    for index, row in df.iterrows():
        now_day_of_week = row['day_of_week']
        # 若 day_of_week 当前天减上一天不是1 或 -6，说明有跳跃
        while (last_day_of_week != 0) and (now_day_of_week - last_day_of_week != 1) and (now_day_of_week - last_day_of_week != -6):
            if last_day_of_week == 7:
                last_day_of_week = 1
            else:
                last_day_of_week += 1
            modi_df = modi_df.append({'day_of_week': last_day_of_week, 'cnt': 0}, ignore_index=True) 
        modi_df = modi_df.append(row, ignore_index=True)
        last_day_of_week = now_day_of_week
    df_len = modi_df.shape[0]
    if df_len != day:
        #print df_len
        #开头部分有缺失
        if modi_df.iloc[0]['day_of_week'] != 3:
            now_day_of_week = modi_df.iloc[0]['day_of_week']
            last_day_of_week = 3
            above = pd.DataFrame([[last_day_of_week,0,np.nan]],columns=['day_of_week','cnt','date'])
            last_day_of_week += 1
            while last_day_of_week != now_day_of_week:
                above = above.append({'day_of_week': last_day_of_week, 'cnt': 0}, ignore_index=True)
                if last_day_of_week == 7:
                    last_day_of_week = 1
                else:
                    last_day_of_week += 1
            modi_df = above.append(modi_df)
        df_len = modi_df.shape[0]     
        assert (modi_df.iloc[0]['day_of_week'] == 3)
        #结尾缺失
        #if (day- df_len)%7 == 0 or modi_df.iloc[0]['day_of_week'] == 3:
        if df_len != day:
            day_num = day-df_len
            #print 'day_num:',day_num
            #填充滿
            last_day_of_week = modi_df.iloc[-1]['day_of_week']
            while (day_num>0):
                if last_day_of_week == 7:
                    last_day_of_week =1
                else:
                    last_day_of_week += 1
                day_num -= 1
                modi_df = modi_df.append({'day_of_week': last_day_of_week, 'cnt': 0}, ignore_index=True)
                #print 'modi_df.shape:',modi_df.shape         
    return modi_df

In [97]:
def run_based_brand(brand,d):
    #brand:品牌
    #d:brand对应的数据
    train = d
    new_df = train.drop(['day_of_week', 'brand'],axis=1).groupby('date').sum()
    new_df['date'] = new_df.index
    new_df = pd.merge(new_df, train[['date','day_of_week']].drop_duplicates(), how='left', on='date')
    #print new_df.shape
    #new_df.to_csv('./data/train_group.txt', sep='\t')
    #填充缺失值
    modi_df = fill_missing(new_df)
    print '填充缺失值：',modi_df.shape
    # 改变 cnt 列的值属性为数值型
    modi_df.cnt = pd.to_numeric(modi_df.cnt)
    
    #均值按星期填充
    for i in range(1, 8):
        # 对每个 day_of_week 依次填充
        day_of_week_mean = int(new_df[new_df.day_of_week == i]['cnt'].mean())
        modi_df.loc[modi_df.day_of_week == i, 'cnt']= modi_df[modi_df.day_of_week == i].cnt.mask(modi_df.cnt == 0, day_of_week_mean)
    print '按星期均值填充：，',modi_df.shape 
    
    modi_df['ds'] = pd.date_range(start = '1/2/2013', periods=1192)
    prophet_train = modi_df.loc[:,['ds', 'cnt']]
    prophet_train.rename(columns = {'cnt':'y'}, inplace = True)
    with open('./data/holidays.pkl', 'rb') as rf:
        holidays = pkl.load(rf)
    m = Prophet(holidays = holidays)

    #测试
    train_d = prophet_train[:1000]
    test_d = prophet_train[1000:]
    test_d.reset_index(drop=True,inplace=True)
    future = pd.DataFrame({'ds':pd.date_range(start='9/29/2015',periods=192)})
    m.fit(train_d)
    forecast = m.predict(future)
    return forecast,test_d
        

In [109]:
forecast_list = []
test_list = []
for num,d in df_group:
    forecast,test_d = run_based_brand(num,d)
    forecast_list.append(forecast)
    test_list.append(test_d)

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


填充缺失值： (1192, 3)
按星期均值填充：， (1192, 3)


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


填充缺失值： (1192, 3)
按星期均值填充：， (1192, 3)


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


填充缺失值： (1192, 3)
按星期均值填充：， (1192, 3)


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


填充缺失值： (1192, 3)
按星期均值填充：， (1192, 3)


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


填充缺失值： (1192, 3)
按星期均值填充：， (1192, 3)


In [104]:
forecast_all = pd.DataFrame(columns=['ds','yhat'])
forecast_all['ds'] = forecast_list[0].ds
forecast_all['yhat'] = sum([x.yhat for x in forecast_list])
forecast_all.head(5)

,ds,yhat
0,2015-09-29,2783.334203
1,2015-09-30,2415.998267
2,2015-10-01,2028.496407
3,2015-10-02,2331.246033
4,2015-10-03,960.269340


In [110]:
test_all = pd.DataFrame(columns=['ds','y'])
test_all['ds'] = test_list[0].ds
test_all['y'] = sum([x.y for x in test_list])
test_all.head(5)

,ds,y
0,2015-09-29,3478.0
1,2015-09-30,1972.0
2,2015-10-01,2764.0
3,2015-10-02,2165.0
4,2015-10-03,598.0


In [111]:
test_all.describe()

,y
count,192.000000
mean,2217.234375
std,1034.932348
min,309.000000
25%,1752.000000
50%,2119.500000
75%,2706.500000
max,5157.000000


In [112]:
evaluate_model(forecast_all,test_all)

336797.7428839639